In [184]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df1=pd.read_csv("ChampsElysées.csv",sep=";")
df2=pd.read_csv("Convention.csv",sep=";")
df3=pd.read_csv("St_peres.csv",sep=";")

df_tot=pd.concat([df1,df2,df3],ignore_index=True)
df_tot.head()

,Identifiant arc,Libelle,Date et heure de comptage,Débit horaire,Taux d'occupation,Etat trafic,Identifiant noeud amont,Libelle noeud amont,Identifiant noeud aval,Libelle noeud aval,Etat arc,Date debut dispo data,Date fin dispo data,geo_point_2d,geo_shape
0,4264,AV_Champs_Elysees,2023-02-01T23:00:00+01:00,764.0,11.28000,Fluide,2294,Av_Champs_Elysees-Washington,2293,Av_Champs_Elysees-Berri,Invalide,2005-01-01,2019-06-01,"48.87153587897717, 2.3017227924560615","{""coordinates"": [[2.3009951475338766, 48.87177..."
1,4264,AV_Champs_Elysees,2023-02-01T22:00:00+01:00,819.0,16.52056,Pré-saturé,2294,Av_Champs_Elysees-Washington,2293,Av_Champs_Elysees-Berri,Invalide,2005-01-01,2019-06-01,"48.87153587897717, 2.3017227924560615","{""coordinates"": [[2.3009951475338766, 48.87177..."
2,4264,AV_Champs_Elysees,2023-02-01T20:00:00+01:00,1082.0,19.98722,Pré-saturé,2294,Av_Champs_Elysees-Washington,2293,Av_Champs_Elysees-Berri,Invalide,2005-01-01,2019-06-01,"48.87153587897717, 2.3017227924560615","{""coordinates"": [[2.3009951475338766, 48.87177..."
3,4264,AV_Champs_Elysees,2023-02-01T18:00:00+01:00,1027.0,23.27334,Pré-saturé,2294,Av_Champs_Elysees-Washington,2293,Av_Champs_Elysees-Berri,Invalide,2005-01-01,2019-06-01,"48.87153587897717, 2.3017227924560615","{""coordinates"": [[2.3009951475338766, 48.87177..."
4,4264,AV_Champs_Elysees,2022-12-07T00:00:00+01:00,528.0,68.14334,Bloqué,2294,Av_Champs_Elysees-Washington,2293,Av_Champs_Elysees-Berri,Invalide,2005-01-01,2019-06-01,"48.871535879911605, 2.3017227924560624","{""coordinates"": [[2.300995147533877, 48.871777..."


In [185]:
df_tot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27930 entries, 0 to 27929
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Identifiant arc            27930 non-null  int64  
 1   Libelle                    27930 non-null  object 
 2   Date et heure de comptage  27930 non-null  object 
 3   Débit horaire              27518 non-null  float64
 4   Taux d'occupation          27498 non-null  float64
 5   Etat trafic                27930 non-null  object 
 6   Identifiant noeud amont    27930 non-null  int64  
 7   Libelle noeud amont        27930 non-null  object 
 8   Identifiant noeud aval     27930 non-null  int64  
 9   Libelle noeud aval         27930 non-null  object 
 10  Etat arc                   27930 non-null  object 
 11  Date debut dispo data      27930 non-null  object 
 12  Date fin dispo data        27930 non-null  object 
 13  geo_point_2d               27930 non-null  obj

Taux d’occupation (en pourcentage de temps d'occupation de la station de mesure par des véhicules sur l’heure).
Selon la valeur de K, on peut qualifier l’état de trafic au point de mesure :
- Fuide : 1
- Pré-saturé : 2
- Saturé : 3
- Bloqué : 4
- Inconnu: 0

État ouvert ou non (barré, inconnu ou invalide) à la circulation de l’arc :
- Inconnu : 0
- Ouvert : 1
- Barré : 2
- Invalide : 3

In [186]:
etat_trafic={"Fluide":1,"Pré-saturé":2,"Saturé":3,"Bloqué":4,"Inconnu":0}
etat_arc={"Ouvert":1,"Barré":2,"Invalide":3,"Inconnu":0}

In [187]:
df_train=pd.DataFrame(data={"id arc":df_tot["Identifiant arc"], "id noeud amont": df_tot["Identifiant noeud amont"],"id noeud aval":df_tot["Identifiant noeud aval"],"débit horaire":df_tot["Débit horaire"],"taux occupation":df_tot["Taux d\'occupation"]})
df_train["year"]=df_tot["Date et heure de comptage"].apply(lambda time: time[:4])
df_train["month"]=df_tot["Date et heure de comptage"].apply(lambda time: time[5:7])
df_train["day"]=df_tot["Date et heure de comptage"].apply(lambda time: time[8:10])
df_train["hour"]=df_tot["Date et heure de comptage"].apply(lambda time: time[11:13])
df_train["etat trafic"]=df_tot["Etat trafic"].apply(lambda etat:etat_trafic[etat])
df_train["etat arc"]=df_tot["Etat arc"].apply(lambda etat:etat_arc[etat])

df_train=df_train[["year","month","day","hour","id arc","id noeud amont","id noeud aval","etat trafic","etat arc","débit horaire","taux occupation"]]

df_train


,year,month,day,hour,id arc,id noeud amont,id noeud aval,etat trafic,etat arc,débit horaire,taux occupation
0,2023,02,01,23,4264,2294,2293,1,3,764.0,11.28000
1,2023,02,01,22,4264,2294,2293,2,3,819.0,16.52056
2,2023,02,01,20,4264,2294,2293,2,3,1082.0,19.98722
3,2023,02,01,18,4264,2294,2293,2,3,1027.0,23.27334
4,2022,12,07,00,4264,2294,2293,4,3,528.0,68.14334
...,...,...,...,...,...,...,...,...,...,...,...
27925,2023,02,01,01,5672,2973,2937,1,3,83.0,1.13945
27926,2023,08,01,01,5672,2973,2937,1,3,61.0,0.74666
27927,2023,08,01,04,5672,2973,2937,1,3,16.0,0.70500
27928,2023,08,01,03,5672,2973,2937,1,3,20.0,0.39444


In [188]:
df_train=df_train.dropna()

In [189]:
df_train.isna().sum()

year               0
month              0
day                0
hour               0
id arc             0
id noeud amont     0
id noeud aval      0
etat trafic        0
etat arc           0
débit horaire      0
taux occupation    0
dtype: int64

In [190]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import model_selection
from sklearn import preprocessing
from sklearn import neighbors, metrics


In [191]:
X = df_train.iloc[:, :-2]
y1 = df_train[["débit horaire"]]
y2 = df_train[["taux occupation"]]


X_train, X_test, y1_train, y1_test = train_test_split(X, y1, test_size= 0.2, random_state=1)
X_train, X_test, y2_train, y2_test = train_test_split(X, y1, test_size= 0.2, random_state=1)



In [192]:
params_rf = {"max_depth": [20],
                "min_samples_split": [10],
                "max_leaf_nodes": [175],
                "min_samples_leaf": [5],
                "n_estimators": [250],
                 "max_features": ["sqrt"],
                }

rf = RandomForestRegressor(random_state=1)

rf_cv = GridSearchCV(rf, params_rf, cv = 3, n_jobs = -1, verbose = False)

rf_cv.fit(X_train.values, np.ravel(y1_train))

rf = rf_cv.best_estimator_


neigh=KNeighborsRegressor()
neigh.fit(X_train.values, np.ravel(y1_train))


# Fixer les valeurs des hyperparamètres à tester
param_grid = {'n_neighbors':[2,3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]}

# Choisir un score à optimiser, ici l'accuracy (proportion de prédictions correctes)
score = 'accuracy'

std_scale = preprocessing.StandardScaler().fit(X_train)
X_train_std = std_scale.transform(X_train)
X_test_std = std_scale.transform(X_test)

# Créer un classifieur kNN avec recherche d'hyperparamètre par validation croisée
clf = model_selection.GridSearchCV(
    neighbors.KNeighborsClassifier(), # un classifieur kNN
    param_grid,     # hyperparamètres à tester
    cv=5,           # nombre de folds de validation croisée
    scoring=score   # score à optimiser
)

# Optimiser ce classifieur sur le jeu d'entraînement
clf.fit(X_train_std, y1_train)

# Afficher le(s) hyperparamètre(s) optimaux
print("Meilleur(s) hyperparamètre(s) sur le jeu d'entraînement:")
print(clf.best_params_)
opti_neigh=KNeighborsRegressor(n_neighbors=clf.best_params_["n_neighbors"])
opti_neigh.fit(X_train_std, np.ravel(y1_train))



/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:230: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be acce

Meilleur(s) hyperparamètre(s) sur le jeu d'entraînement:
{'n_neighbors': 2}


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:230: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


KNeighborsRegressor(n_neighbors=2)

In [193]:
print(rf.score(X_test,y1_test))
print(neigh.score(X_test,y1_test))
print(opti_neigh.score(std_scale.transform(X_test),y1_test))


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/base.py:434: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/base.py:434: UserWarning: X has feature names, but KNeighborsRegressor was fitted without feature names
  warnings.warn(


0.8968514207090257
0.9159709539933226
0.924644117826546


In [194]:
params_rf = {"max_depth": [20],
                "min_samples_split": [10],
                "max_leaf_nodes": [175],
                "min_samples_leaf": [5],
                "n_estimators": [250],
                 "max_features": ["sqrt"],
                }

rf = RandomForestRegressor(random_state=1)

rf_cv = GridSearchCV(rf, params_rf, cv = 3, n_jobs = -1, verbose = False)

rf_cv.fit(X_train.values, np.ravel(y2_train))

rf = rf_cv.best_estimator_

neigh=KNeighborsRegressor()
neigh.fit(X_train.values, np.ravel(y2_train))

# Fixer les valeurs des hyperparamètres à tester
param_grid = {'n_neighbors':[2,3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]}

# Choisir un score à optimiser, ici l'accuracy (proportion de prédictions correctes)
score = 'accuracy'

std_scale = preprocessing.StandardScaler().fit(X_train)
X_train_std = std_scale.transform(X_train)
X_test_std = std_scale.transform(X_test)

# Créer un classifieur kNN avec recherche d'hyperparamètre par validation croisée
clf = model_selection.GridSearchCV(
    neighbors.KNeighborsClassifier(), # un classifieur kNN
    param_grid,     # hyperparamètres à tester
    cv=5,           # nombre de folds de validation croisée
    scoring=score   # score à optimiser
)

# Optimiser ce classifieur sur le jeu d'entraînement
clf.fit(X_train_std, y2_train)

# Afficher le(s) hyperparamètre(s) optimaux
print("Meilleur(s) hyperparamètre(s) sur le jeu d'entraînement:")
print(clf.best_params_)
opti_neigh=KNeighborsRegressor(n_neighbors=clf.best_params_["n_neighbors"])
opti_neigh.fit(X_train_std, np.ravel(y2_train))


print(rf.score(X_test,y2_test))
print(neigh.score(X_test,y2_test))
print(opti_neigh.score(std_scale.transform(X_test),y2_test))




/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:230: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be acce

Meilleur(s) hyperparamètre(s) sur le jeu d'entraînement:
{'n_neighbors': 2}
0.8968514207090257


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/base.py:434: UserWarning: X has feature names, but KNeighborsRegressor was fitted without feature names
  warnings.warn(


0.9159709539933226
0.924644117826546
